In [ ]:
import os
import sys
sys.path.insert(0,'/code/')

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as T
from torchvision.datasets import ImageNet, CIFAR100, CIFAR10, STL10, ImageFolder
import pandas as pd
from torchvision.utils import save_image

import multiprocessing as mp
from tqdm.notebook import tqdm

from datasets.imagenette import ImageNette
from metrics.hamming_distance import hamming_distance
from adv3_robustness_check.py import get_rotation_angles, get_translation_tuples
from utils.transforms import Rotate, Translate, ChangeSaturation, ChangeHue, ChangeContrast, ChangeBrightness, \
    JpegCompression, HorizontalFlipping, BlackBorder, CenterCrop, VerticalFlipping
from scipy.interpolate import griddata

sns.set_style("ticks", {'axes.grid': True})

In [ ]:
DATASET = 'imagenet_train'
MAX_ROT_ANGLE = 64
ROT_LOG_BASE = 2
ROT_STEPS = 7
MAX_TRANS = 64
TRANS_LOG_BASE = 2
TRANS_STEPS = 7

HASH_DIR = f'dataset_hashes/{DATASET}'
PLOT_DIR = f'robustness_plots/{DATASET}'
if not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)
EXAMPLE_IMG_CLASS = 'flamingo'
EXAMPLE_IMG_IDX = 2
EXAMPLE_IMG_DIR = f'robustness_plots/{DATASET}/example_imgs'
if not os.path.exists(EXAMPLE_IMG_DIR):
    os.makedirs(EXAMPLE_IMG_DIR)

In [ ]:
def get_dataset(dataset_name: str):
    transforms = T.Compose([
        T.Resize((360, 360)),
        T.ToTensor()
    ])
    if dataset_name.lower() == 'stl10':
        dataset = STL10(root='data', split='train', download=True, transform=transforms)
    elif dataset_name.lower() == 'imagenette':
        dataset = ImageNette(root='data', train=True, download=True, transform=transforms)
    elif dataset_name.lower() == 'imagenette_val':
        dataset = ImageNette(root='data', train=False, download=True, transform=transforms)
    elif dataset_name.lower() == 'cifar10':
        dataset = CIFAR10(root='data', train=True, download=True, transform=transforms)
    elif dataset_name.lower() == 'cifar100':
        dataset = CIFAR100(root='data', train=True, download=True, transform=transforms)
    elif dataset_name.lower() == 'imagenet_test':
        dataset = ImageFolder(root='data/ILSVRC2012_test', transform=transforms)
    elif dataset_name.lower() == 'imagenet_train':
        dataset = ImageNet(root='data/ILSVRC2012', split='train', transform=transforms)
    elif dataset_name.lower() == 'imagenet_val':
        dataset = ImageNet(root='data/ILSVRC2012', split='val', transform=transforms)
    else:
        raise RuntimeError(f'Dataset with name {dataset_name} was not found.')

    return dataset
    

def get_hashes_from_csv(file_path):
    df = pd.read_csv(file_path)
    bin_hashes = []
    for bit_string in df['hash_bin']:
        bin_hashes.append(list(bit_string))
    bin_hashes = np.array(bin_hashes, dtype=int)

    return bin_hashes


def plot_corr_mat(df, min_corr_val=None, max_corr_val=None):
    corr_mat = df.corr()
    corr_values = np.sort(np.unique(corr_mat.values.flatten()))[:-1]
    print(f'Min Corr. Value: {corr_values.min()}')
    print(f'Max Corr. Value: {corr_values.max()}')
    sns.heatmap(
        corr_mat,
        cmap=sns.diverging_palette(250, 15, as_cmap=True),
        vmax=max_corr_val if max_corr_val else corr_values.max(),
        center=0,
        vmin=min_corr_val if min_corr_val else corr_values.min()
    )
    plt.show()


def get_augmented_hashes_and_hamming_dist(filenames, augment_values, original_hashes, return_bin_hashes=False, num_processes=2):
    return_hash_dict = {}
    return_hamming_dict = {}

    with tqdm(total=len(filenames)) as pbar:
        with mp.Pool(num_processes) as pool:
            for i, bin_hashes in enumerate(pool.imap(get_hashes_from_csv, filenames)):
                return_hash_dict[augment_values[i]] = bin_hashes
                return_hamming_dict[augment_values[i]] = hamming_distance(torch.tensor(bin_hashes), original_hashes).numpy()
                pbar.update()

    if return_bin_hashes:
        return return_hash_dict, return_hamming_dict

    return return_hamming_dict


def print_mean_and_std_for_keys(given_dict):
    for key in given_dict.keys():
        print(f'Mean Hamming Distance for {key}: {given_dict[key].mean()}')
        print(f'Standard Deviation Hamming Distance for {key}: {given_dict[key].std()}')
        

def check_for_non_altered_hashes(given_dict, key, return_indices=False):
    non_altered_hash_indices = (given_dict[key] == 0).nonzero()[0]
    print(f'{len(non_altered_hash_indices)} ({float(len(non_altered_hash_indices)) / float(len(given_dict[key])):5.4f}) hashes were not altered when using key {key}')

    if return_indices:
        return non_altered_hash_indices


def plot_example_img_with_transformation(dataset, img_idx, img_class=None, transformation=None, file_path=None):
    if transformation is not None:
        transforms = T.Compose([
            T.Resize((360, 360)),
            T.ToTensor(),
            transformation
        ])
        dataset.transform = transforms
    base_index = 0
    if img_class is not None:
        base_index = np.argmax(np.array(dataset.targets) == dataset.class_to_idx[img_class])
    img_idx += base_index
    print(f'{dataset.classes[dataset[img_idx][1]][0]}:')
    image = dataset[img_idx][0].permute(1,2,0).numpy()
    plt.imshow(image)
    plt.show()

    if file_path is not None:
        save_image(dataset[img_idx][0], file_path)

    dataset.transform = T.Compose([
            T.Resize((360, 360)),
            T.ToTensor()
    ])
    

In [ ]:
dataset = get_dataset(DATASET)

plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, file_path=os.path.join(EXAMPLE_IMG_DIR, 'original.png'))

In [ ]:
bin_hashes_orig = torch.tensor(get_hashes_from_csv(os.path.join(HASH_DIR, f'{DATASET}_original.csv')))

# Rotation Robustness

In [ ]:
# get the rotation angles
angles = get_rotation_angles(MAX_ROT_ANGLE, ROT_LOG_BASE, ROT_STEPS)

In [ ]:
rot_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'rotation', f'{DATASET}_rotation_{angle}.csv') for angle in angles],
    angles,
    bin_hashes_orig,
    num_processes=15
)

In [ ]:
df = pd.DataFrame(rot_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Angle', 'value': 'Normalized Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Angle", y="Normalized Hamming Distance", ci='sd')
fig.set_xlabel("Angle", fontsize = 21, fontweight='bold')
fig.set_ylabel("Norm. Hamming Distance", fontsize = 21, fontweight='bold')
plt.xticks([-64, -32, -16,-4, 4, 16, 32, 64])
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.set(ylim=(-0.025, 0.55))
fig.set_yticklabels(fig.get_yticks(), size = 18, fontweight='bold')
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')
plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_rotation_robustness.pdf')

In [ ]:
check_for_non_altered_hashes(rot_hamming, angles[6])
check_for_non_altered_hashes(rot_hamming, angles[8])
check_for_non_altered_hashes(rot_hamming, angles[4])
check_for_non_altered_hashes(rot_hamming, angles[10])
check_for_non_altered_hashes(rot_hamming, angles[0])

# plot the single image that does not change
idx = check_for_non_altered_hashes(rot_hamming, angles[14], return_indices=True)[0]
print(f'{dataset.classes[dataset[idx][1]][0]}:')
plt.imshow(dataset[idx][0].permute(1,2,0).numpy())
plt.show()

In [ ]:
print_mean_and_std_for_keys(rot_hamming)

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=Rotate(angles[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'rotation.png'))

# Translation Robustness

In [ ]:
translations = get_translation_tuples(MAX_TRANS, TRANS_LOG_BASE, TRANS_STEPS)
trans_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'translation', f'{DATASET}_translation_{trans[0]}_{trans[1]}.csv') for trans in translations],
    translations,
    bin_hashes_orig,
    num_processes=24
)
# add the hamming distance without any translation
trans_hamming[(0, 0)] = np.zeros_like(trans_hamming[(1, 1)])

In [ ]:
df = pd.DataFrame(trans_hamming)
df = pd.DataFrame(df.mean()).transpose()
df = df.melt()
sns.set_style("ticks", {'axes.grid': False})
x_values = df['variable_0'].to_numpy()
y_values = df['variable_1'].to_numpy()
z_values = df['value'].to_numpy()
xi = np.linspace(0, x_values.max(), 67, endpoint=True)[None, :]
yi = np.linspace(0, y_values.max(), 67, endpoint=True)[:, None]
scipy_linear = griddata((x_values, y_values), z_values, (xi, yi), rescale=True)
sns.set(rc={'figure.figsize':(8,6)})
cmap = sns.cubehelix_palette(as_cmap=True)
labels = [i if i in [0, 4, 8, 16, 32, 64] else None for i in range(66)]
ax = sns.heatmap(scipy_linear, cmap=cmap, vmax=0.25, vmin=0,
    xticklabels=labels, yticklabels=labels, rasterized=True)
ax.tick_params(left=False, bottom=False)
cbar = ax.collections[0].colorbar
cbar.set_label(label='Norm. Hamming Distance', weight='bold')
cbar.set_ticks([0, 0.05, 0.1, 0.15, 0.2, 0.25])
for label in cbar.ax.get_yticklabels():
    label.set_fontsize(18)
    label.set_fontweight('bold')
ax.scatter(x_values + 0.5, y_values + 0.5, s=15, c='None', edgecolors='yellow')
ax.invert_yaxis()
ax.figure.axes[-1].yaxis.label.set_size(18)
ax.figure.axes[-1].tick_params(labelsize=18)

ax.set_xticklabels(ax.get_xticklabels(), size=18, fontweight='bold') 
ax.set_yticklabels(ax.get_yticklabels(), size=18, fontweight='bold', rotation=0) 

plt.xlabel('Horizontal Translation', fontsize=21, fontweight='bold')
plt.ylabel('Vertical Translation', fontsize=21, fontweight='bold')
plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_translation_robustness.pdf')


In [ ]:
print_mean_and_std_for_keys(trans_hamming)

In [ ]:
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(trans_hamming, translations[-1], return_indices=True)
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=Translate(translations[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'translation.png'))

# Hue Robustness

In [ ]:
hues = list(range(-180, 180, 30))
hue_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'hue', f'{DATASET}_hue_{hue}.csv') for hue in hues],
    hues,
    bin_hashes_orig,
    num_processes=12
)

In [ ]:
hue_hamming[180] = hue_hamming[-180]

In [ ]:
df = pd.DataFrame(hue_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'HSV Angle', 'value': 'Normalized Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="HSV Angle", y="Normalized Hamming Distance", ci='sd')
plt.xticks([-180, -120, -60, 0, 60, 120, 180])
plt.yticks([0.0, 0.05, 0.1, 0.15, 0.2, 0.25])
fig.set_xlabel("Hue Angle", fontsize = 21, fontweight='bold')
fig.set_ylabel("Norm. Hamming Distance", fontsize = 21, fontweight='bold')
fig.set_yticklabels(fig.get_yticks(), size = 18, fontweight='bold')
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')
yticklabels = []
for item in fig.get_yticklabels():
    fmt = '{:0.2f}'
    item.set_text(fmt.format(float(item.get_text())))
    yticklabels += [item]
fig.set_yticklabels(yticklabels)

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_hue_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(hue_hamming)

In [ ]:
check_for_non_altered_hashes(hue_hamming, hues[0])

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=ChangeHue(hues[0]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'hue.png'))

# Brightness Robustness

In [ ]:
brightness_values = list(np.linspace(0, 2, 9, endpoint=True))
brightness_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'brightness', f'{DATASET}_brightness_{brightness}.csv') for brightness in brightness_values],
    brightness_values,
    bin_hashes_orig,
    num_processes=9
)

In [ ]:
df = pd.DataFrame(brightness_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Brightness Factor', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Brightness Factor", y="Norm. Hamming Distance", ci='sd')
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.set(ylabel=None)
fig.set(ylim=(-0.025, 0.55))
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
#fig.set_ylabel(fig.get_ylabel(), fontsize = 21, fontweight='bold')
fig.set_yticklabels([])
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')
plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_brightness_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(brightness_hamming)

In [ ]:
check_for_non_altered_hashes(brightness_hamming, brightness_values[-1])

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=ChangeBrightness(brightness_values[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'brightness.png'))

In [ ]:
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(brightness_hamming, brightness_values[-1], return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

# Contrast Robustness

In [ ]:
contrast_values = list(np.linspace(0, 2, 9, endpoint=True))
contrast_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'contrast', f'{DATASET}_contrast_{contrast}.csv') for contrast in contrast_values],
    contrast_values,
    bin_hashes_orig,
    num_processes=9
)

In [ ]:
df = pd.DataFrame(contrast_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Contrast Factor', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Contrast Factor", y="Norm. Hamming Distance", ci='sd')
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.set(ylabel=None)
fig.set(ylim=(-0.025, 0.55))
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
fig.set_ylabel(fig.get_ylabel(), fontsize = 21, fontweight='bold')
fig.set_yticklabels([])
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_contrast_robustness.pdf')

In [ ]:
check_for_non_altered_hashes(contrast_hamming, contrast_values[-1])
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(contrast_hamming, contrast_values[-1], return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
print_mean_and_std_for_keys(contrast_hamming)

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=ChangeContrast(contrast_values[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'contrast.png'))

# Saturation Robustness

In [ ]:
saturation_values = list(np.linspace(0, 2, 9, endpoint=True))
saturation_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'saturation', f'{DATASET}_saturation_{saturation}.csv') for saturation in saturation_values],
    saturation_values,
    bin_hashes_orig,
    num_processes=9
)

In [ ]:
df = pd.DataFrame(saturation_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Saturation Factor', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Saturation Factor", y="Norm. Hamming Distance", ci='sd')
plt.yticks([0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06])
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
fig.set_ylabel(fig.get_ylabel(), fontsize = 21, fontweight='bold')
fig.set_yticklabels(fig.get_yticks(), size = 18, fontweight='bold')
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')
yticklabels = []
for item in fig.get_yticklabels():
    fmt = '{:0.2f}'
    item.set_text(fmt.format(float(item.get_text())))
    yticklabels += [item]
fig.set_yticklabels(yticklabels)

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_saturation_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(saturation_hamming)

In [ ]:
check_for_non_altered_hashes(saturation_hamming, saturation_values[-1])
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(saturation_hamming, saturation_values[-1], return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=ChangeSaturation(saturation_values[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'saturation.png'))

# Compression Robustness

In [ ]:
compression_values = [100] + list(
    (100 - np.ceil(np.logspace(0, np.log(100) / np.log(1.5), 10, endpoint=True, base=1.5))).clip(0, 100)
)
compression_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'compression', f'{DATASET}_compression_{compression}.csv') for compression in compression_values],
    compression_values,
    bin_hashes_orig,
    num_processes=11
)

In [ ]:
df = pd.DataFrame(compression_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Compression Value', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Compression Value", y="Norm. Hamming Distance", ci='sd')
plt.xticks([0, 40, 64, 78, 87, 92, 100])
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.set(ylabel=None)
fig.set(ylim=(-0.025, 0.55))
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
fig.set_ylabel(None)
fig.set_yticklabels([])
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_compression_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(compression_hamming)

In [ ]:
check_for_non_altered_hashes(compression_hamming, compression_values[5])
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(compression_hamming, compression_values[5], return_indices=True)[-2:]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=JpegCompression(compression_values[5]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'compression.png'))

# Crop Robustness

In [ ]:
crop_values = list(
    filter(
        lambda x: x != 359,
        [360] + list(360 - np.append(np.logspace(0, 7, 8, base=2, endpoint=True, dtype=int), [180]))
    )
)
crop_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'crop', f'{DATASET}_crop_{crop}.csv') for crop in crop_values],
    crop_values,
    bin_hashes_orig,
    num_processes=10
)


In [ ]:
df = pd.DataFrame(crop_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Center Crop Size', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Center Crop Size", y="Norm. Hamming Distance", ci='sd')
plt.xticks([180, 232, 286, 328, 360])
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.set(ylim=(-0.025, 0.55))
fig.invert_xaxis()
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
fig.set_ylabel(fig.get_ylabel(), fontsize = 21, fontweight='bold')
fig.set_yticklabels(fig.get_yticks(), size = 18, fontweight='bold')
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_crop_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(crop_hamming)

In [ ]:
check_for_non_altered_hashes(crop_hamming, crop_values[-2])
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(crop_hamming, crop_values[-2], return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=CenterCrop(crop_values[-2]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'crop.png'))

# Horizontal Flipping Robustness

In [ ]:
hflip_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'hflip', f'{DATASET}_hflip.csv')],
    [0],
    bin_hashes_orig,
    num_processes=1
)

In [ ]:
print(f'Average Hamming Distance: {hflip_hamming[0].mean()}')
print(f'Standard Deviation Hamming Distance: {hflip_hamming[0].std()}')

In [ ]:
check_for_non_altered_hashes(hflip_hamming, 0)
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(hflip_hamming, 0, return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=HorizontalFlipping(), file_path=os.path.join(EXAMPLE_IMG_DIR, 'hflip.png'))

# Vertical Flipping Robustness

In [ ]:
vflip_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'vflip', f'{DATASET}_vflip.csv')],
    [0],
    bin_hashes_orig,
    num_processes=1
)

In [ ]:
print(f'Average Hamming Distance: {vflip_hamming[0].mean()}')
print(f'Standard Deviation Hamming Distance: {vflip_hamming[0].std()}')

In [ ]:
check_for_non_altered_hashes(vflip_hamming, 0)
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(vflip_hamming, 0, return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=VerticalFlipping(), file_path=os.path.join(EXAMPLE_IMG_DIR, 'vflip.png'))

# Downsizing Robustness

In [ ]:
downsizing_values = list(
        filter(
            lambda x: x != 359,
            [360] + list(360 - np.append(np.logspace(0, 7, 8, base=2, endpoint=True, dtype=int), [180]))
        )
    )
downsizing_hamming = get_augmented_hashes_and_hamming_dist(
    [os.path.join(HASH_DIR, 'downsizing', f'{DATASET}_downsizing_{size}.csv') for size in downsizing_values],
    downsizing_values,
    bin_hashes_orig,
    num_processes=9
)

In [ ]:
df = pd.DataFrame(downsizing_hamming)
df = df.melt()
df = df.rename(columns={'variable': 'Image Size', 'value': 'Norm. Hamming Distance'})
sns.set(rc={'figure.figsize':(8,5)})
sns.set_style("whitegrid")
fig = sns.lineplot(data=df, marker='o', markersize=8, x="Image Size", y="Norm. Hamming Distance", ci='sd')
fig.invert_xaxis()
plt.xticks([360, 328, 296, 232, 180])
fig.set(ylabel=None)
fig.set(ylim=(-0.025, 0.55))
fig.set_xlabel(fig.get_xlabel(), fontsize = 21, fontweight='bold')
fig.set_ylabel(None)
fig.set_yticklabels([])
fig.set_xticklabels(fig.get_xticks(), size = 18, fontweight='bold')

plt.tight_layout() 
plt.savefig(f'{PLOT_DIR}/{DATASET}_downsizing_robustness.pdf')

In [ ]:
print_mean_and_std_for_keys(downsizing_hamming)

In [ ]:
check_for_non_altered_hashes(downsizing_hamming, downsizing_values[1])
# plot two images that do not change
idx1, idx2 = check_for_non_altered_hashes(downsizing_hamming, downsizing_values[1], return_indices=True)[:2]
print(f'{dataset.classes[dataset[idx1][1]][0]}:')
plt.imshow(dataset[idx1][0].permute(1,2,0))
plt.show()
print(f'{dataset.classes[dataset[idx2][1]][0]}:')
plt.imshow(dataset[idx2][0].permute(1,2,0))
plt.show()

In [ ]:
plot_example_img_with_transformation(dataset, EXAMPLE_IMG_IDX, img_class=EXAMPLE_IMG_CLASS, transformation=BlackBorder(downsizing_values[-1]), file_path=os.path.join(EXAMPLE_IMG_DIR, 'downsizing.png'))